In [ ]:
! pip install comet-ml>=3.43.2 trl transformers peft bitsandbytes accelerate datasets

In [ ]:
import os
from pathlib import Path
import torch
import warnings
from typing import Any, List, Literal, Optional  # noqa: E402
from datasets import concatenate_datasets, load_dataset  # noqa: E402
from huggingface_hub import HfApi, login  # noqa: E402
from trl import SFTTrainer, SFTConfig  # noqa: E402
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from llm_engineering.settings import settings  # noqa: E402

comet_ml is installed but the Comet API Key is not configured. Please set the `COMET_API_KEY` environment variable to enable Comet logging. Check out the documentation for other ways of configuring it: https://www.comet.com/docs/v2/guides/experiment-management/configure-sdk/#set-the-api-key


In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Use only GPU 0
os.environ["UNSLOTH_SKIP_TOKEN_FIX"] = "1"  # Skip problematic token fixing
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"  # Better memory management
os.environ["COMET_API_KEY"] = settings.COMET_API_KEY  # Set your Comet API key here

# Disable warnings
warnings.filterwarnings("ignore")

# Verify GPU is available
if not torch.cuda.is_available():
    raise RuntimeError("CUDA not available!")

print(f"Using GPU: {torch.cuda.get_device_name(0)}")
print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

Using GPU: Tesla T4
GPU Memory: 15.8 GB


In [ ]:
hf_api = HfApi(token=settings.HF_TOKEN)  # noqa: F821
login(token=settings.HF_TOKEN)  # noqa: F821

In [5]:
class Config:
    def __init__(self):
        self.num_train_epochs = 1
        self.per_device_train_batch_size = 2
        self.learning_rate = 3e-4
        self.dataset_huggingface_workspace = "K-1303"
        self.model_output_huggingface_workspace = "K-1303"
        self.is_dummy = False
        self.finetuning_type = "sft"  # "sft" or "dpo"
        self.output_data_dir = "/content/output"
        self.model_dir = "/content/model"
        self.n_gpus = "1"

In [6]:
args = Config()

In [7]:
print(f"Num training epochs: '{args.num_train_epochs}'")  # noqa
print(f"Per device train batch size: '{args.per_device_train_batch_size}'")  # noqa
print(f"Learning rate: {args.learning_rate}")  # noqa
print(f"Datasets will be loaded from Hugging Face workspace: '{args.dataset_huggingface_workspace}'")  # noqa
print(f"Models will be saved to Hugging Face workspace: '{args.model_output_huggingface_workspace}'")  # noqa
print(f"Training in dummy mode? '{args.is_dummy}'")  # noqa
print(f"Finetuning type: '{args.finetuning_type}'")  # noqa

print(f"Output data dir: '{args.output_data_dir}'")  # noqa
print(f"Model dir: '{args.model_dir}'")  # noqa
print(f"Number of GPUs: '{args.n_gpus}'")  # noqa

Num training epochs: '1'
Per device train batch size: '2'
Learning rate: 0.0003
Datasets will be loaded from Hugging Face workspace: 'K-1303'
Models will be saved to Hugging Face workspace: 'K-1303'
Training in dummy mode? 'False'
Finetuning type: 'sft'
Output data dir: '/content/output'
Model dir: '/content/model'
Number of GPUs: '1'


In [8]:
# Load model function

def load_model(
    model_name: str,
    max_seq_length: int,
    load_in_4bit: bool,
    lora_rank: int,
    lora_alpha: int,
    lora_dropout: float,
    target_modules: List[str],
    chat_template: str,
) -> tuple:
    # Configure 4-bit quantization
    if load_in_4bit:
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16,
            bnb_4bit_use_double_quant=True,
        )
    else:
        bnb_config = None

    # Load model
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True,
        torch_dtype=torch.bfloat16 if not load_in_4bit else None,
    )

    # Prepare model for k-bit training if using quantization
    if load_in_4bit:
        model = prepare_model_for_kbit_training(model)

    # Configure LoRA
    peft_config = LoraConfig(
        r=lora_rank,
        lora_alpha=lora_alpha,
        lora_dropout=lora_dropout,
        target_modules=target_modules,
        bias="none",
        task_type="CAUSAL_LM",
    )

    # Apply PEFT
    model = get_peft_model(model, peft_config)

    # Print trainable parameters
    model.print_trainable_parameters()

    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(
        model_name,
        trust_remote_code=True,
    )

    # Set padding token if not present
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    # Apply chat template
    if chat_template == "chatml":
        tokenizer.chat_template = "{% for message in messages %}{{'<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>' + '\n'}}{% endfor %}{% if add_generation_prompt %}{{ '<|im_start|>assistant\n' }}{% endif %}"

    return model, tokenizer

In [22]:
def inference(
    model: Any,
    tokenizer: Any,
    prompt: str = "Write a paragraph to introduce supervised fine-tuning.",
    max_new_tokens: int = 256,
) -> None:
    from transformers import TextStreamer


    # Set model to evaluation mode
    model.eval()

    try:
        # Check if model has mixed dtypes
        dtypes = set(p.dtype for p in model.parameters())
        if len(dtypes) > 1 or torch.float32 in dtypes:
            print(f"Warning: Model has mixed dtypes {dtypes}. Converting to bfloat16...")
            if torch.cuda.is_bf16_supported():
                model = model.to(dtype=torch.bfloat16)
            else:
                model = model.to(dtype=torch.float16)
    except Exception as e:
        print(f"Could not check/fix dtypes: {e}")

    # Format the prompt
    message = alpaca_template.format(prompt, "")
    inputs = tokenizer([message], return_tensors="pt")

    # Move inputs to the same device as model
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    # Create text streamer for real-time output
    text_streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

    # Generate with the model
    with torch.no_grad():
        _ = model.generate(
            **inputs,
            streamer=text_streamer,
            max_new_tokens=max_new_tokens,
            use_cache=True,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )

In [25]:
# Save model function

def save_model(
    model: Any,
    tokenizer: Any,
    output_dir: str,
    push_to_hub: bool = False,
    repo_id: Optional[str] = None
):
    """
    Save the full merged model (LoRA weights merged with base model)
    """
    print("Merging LoRA weights with base model...")

    # Merge LoRA adapter weights with the base model
    model = model.merge_and_unload()

    print(f"Saving merged model to '{output_dir}'...")

    # Save the merged model in 16-bit precision
    model.save_pretrained(
        output_dir,
        safe_serialization=True,  # Save as safetensors format
    )
    tokenizer.save_pretrained(output_dir)

    print(f"Model saved successfully to '{output_dir}'")

    if push_to_hub and repo_id:
        print(f"Pushing merged model to Hugging Face Hub: '{repo_id}'")
        model.push_to_hub(
            repo_id,
            safe_serialization=True,
        )
        tokenizer.push_to_hub(repo_id)
        print(f"Model pushed successfully to '{repo_id}'")

# Supervised Fine Tuning(LORA)

In [11]:
# Template for training data

alpaca_template = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{}

### Response:
{}"""

In [ ]:
base_model_name = "unsloth/Meta-Llama-3.1-8B"

In [13]:
output_dir_sft = Path(args.model_dir) / "output_sft"

In [ ]:
sft_output_model_repo_id = f"{args.model_output_huggingface_workspace}/TwinLlama-3.1-8B-KD"

In [15]:
# Fine-tuning function
def finetune(
    finetuning_type: Literal["sft", "dpo"],
    model_name: str,
    output_dir: str,
    dataset_huggingface_workspace: str,
    max_seq_length: int = 2048,
    load_in_4bit: bool = False,
    lora_rank: int = 32,
    lora_alpha: int = 32,
    lora_dropout: float = 0.0,
    target_modules: List[str] = ["q_proj", "k_proj", "v_proj", "up_proj", "down_proj", "o_proj", "gate_proj"],
    chat_template: str = "chatml",
    learning_rate: float = 3e-4,
    num_train_epochs: int = 3,
    per_device_train_batch_size: int = 2,
    gradient_accumulation_steps: int = 8,
    beta: float = 0.5,  # Only for DPO
    is_dummy: bool = False,
) -> tuple:
    print("--Run QLORA--")
    print(load_in_4bit)

    model, tokenizer = load_model(
        model_name,
        max_seq_length,
        load_in_4bit,
        lora_rank,
        lora_alpha,
        lora_dropout,
        target_modules,
        chat_template
    )

    # Print trainable parameters
    model.print_trainable_parameters()

    EOS_TOKEN = tokenizer.eos_token
    print(f"Setting EOS_TOKEN to {EOS_TOKEN}")

    if is_dummy is True:
        num_train_epochs = 1
        print(f"Training in dummy mode. Setting num_train_epochs to '{num_train_epochs}'")
        print(f"Training in dummy mode. Reducing dataset size to '400'.")

    def format_samples_sft(examples):
        text = []
        for instruction, output in zip(examples["instruction"], examples["output"], strict=False):
            message = alpaca_template.format(instruction, output) + EOS_TOKEN
            text.append(message)
        return {"text": text}

    # Load datasets
    dataset1 = load_dataset(f"{dataset_huggingface_workspace}/llmtwin", split="train")
    dataset2 = load_dataset("mlabonne/FineTome-Alpaca-100k", split="train[:1000]")
    dataset = concatenate_datasets([dataset1, dataset2])

    if is_dummy:
        try:
            dataset = dataset.select(range(400))
        except Exception:
            print("Dummy mode active. Failed to trim the dataset to 400 samples.")

    print(f"Loaded dataset with {len(dataset)} samples.")

    dataset = dataset.map(format_samples_sft, batched=True, remove_columns=dataset.column_names)
    dataset = dataset.train_test_split(test_size=0.05)

    print("Training dataset example:")
    print(dataset["train"][0])

    # Check if bfloat16 is supported
    is_bfloat16_supported = torch.cuda.is_available() and torch.cuda.is_bf16_supported()

    # Create SFTTrainer
    trainer = SFTTrainer(
        model=model,
        args=SFTConfig(
            learning_rate=learning_rate,
            num_train_epochs=num_train_epochs,
            per_device_train_batch_size=per_device_train_batch_size,
            gradient_accumulation_steps=gradient_accumulation_steps,
            fp16=not is_bfloat16_supported,
            bf16=is_bfloat16_supported,
            logging_steps=10,
            optim="adamw_8bit",
            weight_decay=0.01,
            lr_scheduler_type="linear",
            per_device_eval_batch_size=per_device_train_batch_size,
            warmup_steps=1,
            output_dir=output_dir,
            report_to="comet_ml",
            seed=0,
            save_strategy="epoch",
            dataloader_pin_memory=False,
            eval_strategy="epoch",
            save_total_limit=2,
            packing=True,
            gradient_checkpointing=True,
            group_by_length=True,
            dataloader_num_workers=2
        ),
        train_dataset=dataset["train"],
        eval_dataset=dataset["test"],
        processing_class=tokenizer
    )

    trainer.train()

    return model, tokenizer

In [16]:
model, tokenizer = finetune(
            finetuning_type="sft",
            model_name=base_model_name,
            output_dir=str(output_dir_sft),
            dataset_huggingface_workspace=args.dataset_huggingface_workspace,
            num_train_epochs=args.num_train_epochs,
            per_device_train_batch_size=args.per_device_train_batch_size,
            load_in_4bit=True,
            learning_rate=args.learning_rate,
        )

--Run QLORA--
True


config.json:   0%|          | 0.00/947 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

Tokenizing eval dataset:   0%|          | 0/95 [00:00<?, ? examples/s]

Packing eval dataset:   0%|          | 0/95 [00:00<?, ? examples/s]

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch, sklearn, tensorflow, keras.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/kavya-dua/general/d5ee9df6eaa44cfebb57dd23896757d3

COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.


Epoch,Training Loss,Validation Loss,Entropy,Num Tokens,Mean Token Accuracy
1,0.890500,0.784563,0.801221,514768.000000,0.782665


comet_ml is installed but the Comet API Key is not configured. Please set the `COMET_API_KEY` environment variable to enable Comet logging. Check out the documentation for other ways of configuring it: https://www.comet.com/docs/v2/guides/experiment-management/configure-sdk/#set-the-api-key


In [23]:
inference(model, tokenizer)

Supervised fine-tuning is a method for training a model using a labeled dataset. The model is first initialized with pre-trained weights, which are then fine-tuned using the labeled data. This approach is often used in natural language processing (NLP) tasks, where the model is trained to generate text or perform classification on text data.


In [26]:
save_model(model, tokenizer, "model_sft", push_to_hub=True, repo_id=sft_output_model_repo_id)

Merging LoRA weights with base model...
Pushing merged model to Hugging Face Hub: 'K-1303/TwinLlama-3.1-8B-KD'


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...0002-of-00002.safetensors:   2%|2         | 25.1MB / 1.05GB            

  ...0001-of-00002.safetensors:   0%|          | 30.0kB / 4.65GB            

README.md: 0.00B [00:00, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...mpv19yd7eg/tokenizer.json: 100%|##########| 17.2MB / 17.2MB            

Model pushed successfully to 'K-1303/TwinLlama-3.1-8B-KD'
